# DataViz et données géospatiales, cartes

## Rappel historique

![JohnSnow](./Images/JohnSnow.png)

Une des premières études où la dataviz a permis de résoudre un problème important impliquait de la cartographie. Si vous voulez en savoir plus, voici les articles Wikipedia [sur cette épidémie de 1854](https://en.wikipedia.org/wiki/1854_Broad_Street_cholera_outbreak), et [sur John Snow](https://fr.wikipedia.org/wiki/John_Snow).



Voici quelques articles récents qui reviennent sur ces données historiques :

https://blog.wolfram.com/2021/08/03/john-snow-the-birth-of-epidemiology-data-analysis-visualization/

https://archive.cdc.gov/#/details?url=https://www.cdc.gov/csels/dsepd/ss1978/lesson1/section2.html

Un article sous la forme de « data storytelling » de qualité, démontrant les capacités des outils de cartographie de l’éditeur ESRI : https://storymaps.arcgis.com/stories/9fc482aa122849f8af4008c38991000e

Nous allons dans ce cours voir les bases de la mannipulation des données visuospatiales, qui ont un certain nombre de spécificité, et quelques bibliothèques python qui permettent de les manipuler et de réaliser des dataviz sous forme de cartes.

## Geopandas

**Geopandas** ajoute à ```pandas``` une couche capable de traiter l’information spatiale (des *geometries*), et donc de manipuler ces données sous forme de _dataframes_.
* elle intègre la bibliothèque ```fiona``` pour charger différents formats de fichiers geospatiaux (.gpkg, .shp, etc.)
* elle permet de manipuler tant des fichiers de données vectorielles que des _rasters_ (.tif, .asc, etc.)
* elle peut gérer les systèmes de projection _CRS (Coordinates Reference System)_ avec ```pyproj```
* elle intègre la bibliothèque ```shapely``` pour traiter des geometries (point, lignes, polygone) et réaliser des opérations sur celles-ci (calculer des longueurs, des intersections, etc.)
* grâce à ```leaflet``` elle permet de visualiser rapidement des infos spatiales sur des fonds de cartes (meth»de ```.explore()```)


In [ ]:
!pip install geopandas

In [ ]:
!pip show geopandas

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

Comme d’autres bibliothèques, avant la version 1 Geopandas embarquait des jeux de données a vocation de démo, ce n’est plus le cas depuis avril 2024 et la version 1.0 :

In [ ]:
gpd.datasets.available

Les dev de la bibliothèque proposent à la place de télécharger les données cartographiques du site [Natural Earth Data](https://www.naturalearthdata.com/downloads/), qui sont disponibles en 3 niveau de précision : 1:10m (la plus précise), 1:50m et enfin 1:110m. Attention l’archive en 1:10m est nettement plus lourde (>300Mo) que l’archive 1:110m (quelques Mo). 

Téléchargez ces données pour suivre les exemples du cours

La base Natural Earth est constituée de fichiers vectoriels (`.SHP` ou `.geopackage`) ou raster (`.tif`). La différence est essentielle en matière de données géospatiale.

### Fichiers vectoriels et rasters

#### Objets vectoriels
Les fichiers vectoriels sont des fichiers de données qui contiennent nécessairement un champ `geometry`. Ces géométries peuvent être généralement de 3 types :

* des points (`Point`, `MultiPoints`)
* des lignes (`LineString`, `MultiLineString`, `LinearRing`)
* des polygones (`Polygon`, `Multipolygon`)

Il y a aussi un objet `GeometryCollection` qui comme son nom l’indique contient plusieurs gémotries de plusieurs types.

Ces objets sont définis par un certain nombre de points (coordonnées x et y en 2D ou x, y et z en 3D), et de relations entre ces points (lignes, ou arêtes/côtés = *edges*, et *vertices* pour les sommets) lorsque cela a un sens :

![Illustration géométries](./Images/Geometries.png)

Les fichiers contenant des données vectorielles peuvent tout à fait être des flatfiles type `.csv` avec une colone géométrie ou `.xyz` pour les seules données vectorielles. Un autre format très populaire est le [GeoJSON (lien wikipédia)](https://fr.wikipedia.org/wiki/GeoJSON) : les objets définis dans un JSON (ou dictionnaire pour python) contiennent un attribut `"geometry"` qui contient à son tour deux attribut : `"type"` (comme `point`, `linestring`, etc.) et `"coordinates"` qui contient évidemment les coorodnnées des points constituant les géométries. Attention : dans certains cas l’ordre des coordonnées peut différer d’un GeoJSON à l’autre (par ex. lat/long vs. long/lat…)

#### Fichiers raster
Les fichiers raster sont des images bitmaps (constituées de pixels), dont chaque pixel représente une position dans l’espace : une correspondance est établie entre les coordonnées du pixel dans l’image, et les coordonnées de la surface correspondante dans la réalité. En effet, un raster couvre une certaine surface selon une certaine échelle, dont le pixel est une partie. Par exemple l’IGN met à disposition dans sa base [RGE ALTI](https://geoservices.ign.fr/rgealti) des données altimétriques sous format de *dalles* ou *tuiles* (*tiles*) raster : la surface terrestre est découpée selon une grille qui définie des carrés de 1km de côté, chaque dalle peut être occupée par une image de 1000x1000 pixels, 1 pixel couvre donc une surface de 1m2. Ici la grille pour le département du Vaucluse : 

![Exemple dallage IGN](./Images/Dalles-IGN.png)

À chaque pixel d’autres informations numériques sont associées sous forme de *couches*, *bande* ou *canal* (*channel*). De la même manière que dans une image on peut associé à un pixel une valeur de gris, ou 3 couches de couleur, on peut associer des valeurs numériques comme par exemple une altitude dans notre exemple. On peut afficher une telle image :

![Exemple Raster altimetrique IGN](./Images/Raster-IGN.png)

Comme une image couleur RGB est constituée de 3 couches, on peut avoir plusieurs bandes sur un raster. C’est souvent le cas des imagres satellites qui observent dans plusieurs longueurs d’ondes (souvent infrarouge). Par exemple ici depuis le site [Nasa Worldview](https://worldview.earthdata.nasa.gov/) les écarts à la normale de la température des eaux de surface :

![Water temperature abnormality - El Nino - Nasa Worldview](./Images/Nasa-Worldview.png)

Mais on peut ajouter tout type d’information numérique que l’on veut à chaque pixel (code du type d’acquisition, grandeur physique, etc.) Cette notion est à rapprocher de celle de *voxel* en 3D (courante en imagerie médicale, etc.)

La bibliothèque qui permet de manipuler les rasters à bas niveau est [GDAL](https://pypi.org/project/GDAL/) (attention, en Python l’installation peut demander des manipulations spécifiques). C’est une bibliothèque C++ pour des opérations de bas niveau, nous n’en aurons pas spécialement besoin pour une introduction. De manière générale nous nous focaliserons ici sur les données vectorielles.

#### Nuage de points

Il existe une dernière catégorie de données que l’on pourrait qualifier de vectorielles car elle contiennent des géométries dont les coordonnées sont définies (points) mais qui méritent un traitement spécifique : les nuages de points en 3D, généralement obtenus soit par des techniques três spécifiques, par exemple photogrammétre (prise de vue d’un objet depuis plusieurs points de vue), ou Lidar (laser infrarouge). Voir par exemple [le projet Lidar HD de l’IGN](https://geoservices.ign.fr/lidarhd)

Ces données demandent d’utiliser plutôt des bibliothèques spécialisées comme [open3D](https://www.open3d.org/) pour manipuler des objets 3D ou [laspy](https://laspy.readthedocs.io/en/latest/index.html) pour charger des fichiers d’acquisition Lidar au format `.LAS` ou `.LAZ`. Des bibliothèques plus intégrées existent comme [geemap](https://geemap.org/) (qui peut utiliser open3d en backend, et repose sur [Earth Engine](https://earthengine.google.com/)).

#### L’écosystème des SIG

Pour la visualisation et le traitement de ce type de données dans les SIG (Systèmes d’Information Géographiques) ou GIS en anglais, deux grands logiciels sont utilisés : [QGIS](https://qgis.org/) (open source) et [ArcGIS](https://arcgis.com/) (propriétaire, de la société ESRI). Il existe d’autres solutions pour le stockage des données en base comme [PostGIS](https://postgis.net/) (dérivé de PostGreSQL), ou des serveurs pour fournir des flux de données WMS (cartes), WFS (features) ou encore WCS (rasters) comme [GeoServer](https://geoserver.org/). Les SIG sont un monde à part entière.

#### Quelques sources de données

Maintenant que vous disposez d’un panorama sur les données géospatiales pour enrichir votre culture générale, revenons aux données cartographiques 2D et vectorielles les plus courantes.

Les sources de données géospatiales peuvent être nombreuses :

- des sites open data (gouvernementaux ou privés) : IGN avec sa plateforme [Géoservices](https://geoservices.ign.fr/), [Open Topography](https://portal.opentopography.org/datasets), [Nasa Worldview](https://worldview.earthdata.nasa.gov/), etc.
- sur [Kaggle](https://www.kaggle.com/search?q=geospatial+in%3Adatasets) (datasets avec le mot clé geospatial)
- la bibliothèque [Geodataset](https://pypi.org/project/geodataset/) propose une API pour charger des datasets complet pour l’entraînement ou la familiarisation au traitement de données géospatiales.

Jetons un œil à quelques fichiers téléchargés sur Natural Earth.

In [ ]:
worldmap_df = gpd.read_file('data/Natural-Earth-Data_110m_cultural/ne_110m_admin_0_countries.shp')

In [ ]:
worldmap_df.head()

### Les systèmes de coordonnées de référence

In [ ]:
worldmap_df.info()

Une info importante quand on manipule des données géospatiales et notamment des cartes est le _coordinate reference system_ :

In [ ]:
worldmap_df.crs

Ici WGS 84 est le système de coordonnées en longitude/latitude (type GPS). Les valeurs des coordonnées des points sont des long/lat (« 2D ») et altitude (3D), dans un référenciel sphérique. Or nous représentons généralement ces données sur des cartes en 2D : il faut donc réaliser une projection de la 3D vers la 2D, et c’est là que les difficultés émergent.

Selon les systèmes de projection choisi, différentes propriétés géométriques peuvent être altérées ou non (direction/orientations, angles, longueurs, aires…), il faut donc choisir un système de projection dont les effets sur ces caractérsitiques sont compatibles avec notre objectif. Par ailleurs, il faut toujours associer des données dont le CRS est identique.

La projection la plus « simple » est la projection cylindrique de [Mercator](https://fr.wikipedia.org/wiki/Projection_de_Mercator#/media/Fichier:Usgs_map_mercator.svg).
Pour éliminer au maximum les déformations lorsque l’on se focalise sur une zone relativement localisée (par exemple à l’échelle d’un pays) on a recours à des projections coniques « tangentes » au plus près de la zone d’intérêt. Par exemple en France métropolitaine la projection officielle est la [projection conique conforme de Lambert](https://upload.wikimedia.org/wikipedia/commons/7/7b/Lambert_et_mercator_pour_wikipedia.svg?uselang=fr). Attention donc si vous consultez des données ailleurs dans le monde, et plus particulièrement des données pour la France d’outre-mer (Réunion, Martinique, Guadeloupe… etc.) : il faudra utiliser un CRS ad-hoc.

Dans le système Lambert 93, les coordonnées s’expriment en métres, à partir d’un point d’origine situé à Bourges et de coordonnées X = 600 000 mètres et Y = 2 200 000 mètres.

Un CRS est défini par un code, un nom, un système de référence, une origine, des unités de mesures… etc.
Ils sont catalogué dans le registre EPSG (European Petroleum Survey Group). Ainsi le système GPS (WGS 84) porte le code EPSG:4326 (voir exemple ci-dessus avec nos données). Pour la France métropolitaine (Lambert 93), il s’agit de l’EPSG:2154.

### Affichage avec la méthode `.plot()`

Cette méthode permet d’afficher les géométries :

In [ ]:
worldmap_df.plot();

Essayons avec un autre fichier :

In [ ]:
wmpop_df = gpd.read_file(\
        'data/Natural-Earth-Data_110m_cultural/ne_110m_populated_places.shp')

In [ ]:
wmpop_df.head()

Ce fichier contient semble-t-il des informations sur les capitales du monde :

In [ ]:
for c in wmpop_df:
    print(c) # boucle pour afficher la totalité des colonnes

Affichons les géométries :

In [ ]:
wmpop_df.plot();

On peut afficher des données (ici la colonne qui donne l’estimation de la population en 2025) :

In [ ]:
wmpop_df.plot('POP2025', marker='v');

Geopandas dispose d’une méthode `.explore()` qui ajoute un fond de carte et de l’interaction (si vous avez déjà matplotlib installé, installez aussi folium - que nous verrons plus en détail ci-dessous - et mapclassify) :

In [ ]:
# !pip install matplotlib folium mapclassify

wmpop_df.explore()

Cette méthode repose sur Leaflet, une bibliothèque javascript. Il est possible que pour utiliser ̀`leaflet` Jupyter vous demande « Make this Notebook Trusted to load map:File -> Trust Notebook ». Dans Jupyter Lab, il faut faire 
`View` -> `Activate Command Palette`
et dans le champ de recherche taper `trust` vous pourrez choisir Trust Notebook, et confirmer.

On peut aussi superposer 2 tracés à l’aide de matplotlib : 

In [ ]:
fig, ax = plt.subplots()

worldmap_df.plot(ax=ax, color='lightgray')
wmpop_df.plot(ax=ax, marker='v', markersize=3)

plt.title('Villes du monde')
plt.show()

Peut-on isoler une carte de l’Afrique ?
Regardons déjà la colonne « continent » :

In [ ]:
worldmap_df.CONTINENT.unique()

In [ ]:
afriquemap_df = worldmap_df[worldmap_df['CONTINENT']=='Africa'].copy()
afriquemap_df.plot();

Calculons les coordonnées du centre de chaque pays :

In [ ]:
afriquemap_df['CenterX'] = afriquemap_df['geometry'].map(lambda c: c.centroid.x)
afriquemap_df['CenterY'] = afriquemap_df['geometry'].map(lambda c: c.centroid.y)

Affichons maintenant la carte avec le code iso de chaque pays aux centres que nous venons de calculer :

In [ ]:
fig, ax = plt.subplots()
afriquemap_df.plot(ax=ax, color='lightgray', edgecolor='white')

for idx, row in afriquemap_df.iterrows():
    ax.text(row["CenterX"], 
            row["CenterY"],
            row["ISO_A3"], 
            ha="center", 
            va="center", 
            size=8)

plt.title('Codes pays d’Afrique')
plt.show()

Affichons la population estimée pour chaque pays, directement depuis la méthode `.plot()` du dataframe :

In [ ]:
afriquemap_df.plot(column = "POP_EST",
                   cmap='RdBu',
                   legend=True,
                   edgecolor='black', 
                   label='Population estimate per state');

### Exercice

Reproduisez cet exemple dans d’autres régions du monde : explorez les données des fichiers utilisés.

In [ ]:
# votre code ici



## Cartes choroplèthes

Les cartes telles que nous avons produites, des « carte[s] thématique[s] où les régions sont colorées ou remplies d'un motif qui montre une mesure statistique » sont des cartes dites choroplèthes (définition [Wikipédia](https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the)).

Ce sont des cartes très utilisées en dataviz, de nombreux outils permettent de les produire, avec plus ou moins d’interactivité : matplotlib (qu’on peut enrichir de surcouches come [cartopy](https://scitools.org.uk/cartopy/docs/latest/), etc.), [folium](https://python-visualization.github.io/folium/latest/) basée sur [Leaflet.js](https://leafletjs.com/), [plotly](https://plotly.com/python/), etc.

### Cartopy

C’est une surcouche de matplotlib qui peut être utiliser pour manipuler des projections, etc. Son module `feature` contient des données du projet Natural Earth, ce qui permet de les importer sans télécharger nous-même les fichiers du projet à chaque fois. En fait cartopy va les télécharger pour nous, il enverra un warning à chaque fois pour indiquer que quelque chose est téléchargé. On peut inhiber ces warnings avec `warnings.filterwarnings('ignore')` si vous le désirez.

#### Projections

Un exemple qui vous permet de comparer les différentes projections que nous avons évoqués dans la sections précédente, car Cartopy a été pensée pour être construite à l’aide de pyproj autour de ces système de projection (c’est avant tout un outil pédagogique). **ATTENTION** il y a une subtile différence par exemple avec un CRS comme WGS-84 qui n’est précisément pas un système de projection (PRS), mais un système de coordonnées (CRS).

In [ ]:
from cartopy import crs, feature

fig = plt.figure(figsize=(11, 8.5))
ax = plt.axes(projection=crs.PlateCarree(central_longitude=0))
ax.set_title("Projection Plate Carrée")
# Note : sur une projection plate carrée les points sont écartées du même nombre de degrés
ax.coastlines()
ax.add_feature(feature.BORDERS, linewidth=0.5, edgecolor='blue');

Montrons la même carte avec une autre projection, la projection de Mollweide ou *homolographique*, très utilisée pour les planisphères. C’est une projection pseudo-cylindrique qui conserve les aires (mais pas les angles)

In [ ]:
fig = plt.figure(figsize=(11, 8.5))
ax = plt.axes(projection=crs.Mollweide(central_longitude=0))
ax.set_title("Projection de Mollweide")
ax.coastlines()
ax.add_feature(feature.BORDERS, linewidth=0.5, edgecolor='blue');

On peut aussi ajouter une image de fond avec la méthode `ax.stock_img()`

In [ ]:
fig = plt.figure(figsize=(11, 8.5))
ax = plt.axes(projection=crs.Mollweide(central_longitude=0))
ax.set_title("Projection de Mollweide")
ax.coastlines()
ax.add_feature(feature.BORDERS, linewidth=0.5, edgecolor='red')
ax.stock_img();

On peut adopter des point de vue t

In [ ]:
fig = plt.figure(figsize=(8,8))

projection=crs.NearsidePerspective(central_longitude=0,
                                    central_latitude=45,
                                    satellite_height=2500000)
ax = plt.axes(projection=projection)
ax.set_title("Vue de l’espace")
ax.coastlines(resolution='50m')
ax.add_feature(feature.BORDERS, linewidth=0.5, edgecolor='red')
ax.stock_img()
plt.show()

#### Zoom sur une région

On n’a pas toujours besoin d’afficher un panisphère au complet. Pour se focaliser sur une zone nous allons utiliser la méthode `.set_extent()`. Elle prend comme argument les coordonnées des 4 coins de la « fenêtre » qu’on va afficher, dans un CRS donné bien sûr. On peut très bien utiliser une projection différente pour définir `.set_extent()` et pour représenter les données géographiques.

Dans l’exemple suivant on va représenter les données géographiques avec une projection stéréographique (projection conforme – c.-à-d. qui conserve les angles – obtenue en projetant les points de la sphère terrestre sur un plan tangeant en un point défini). Et nous allons définir la « fenêtre » par des longitudes et latitudes limites en est et en ouest, dans une projection plate carrée.

Nous allons : 
* d’abord définir ces limites (dans les 4 directions)
* déterminer le centre de la fenêtre où nous centrerons aussi la projection stéréographique
* définir une résolution pour les données Natural Earth
* déclarer notre figure
* utiliser la méthode `.gridlines` pour placer des repères (méridiens et parallèles)
* utiliser la méthode `.set_extent()` pour définir la « fenêtre »
* et rajouter différents `feature` que nous n’avons pas présentés, ainsi que la manière de les mettre en forme. Attention de fixer la résolution de ces features à égalité avec la résolution définie pour la figure, histoire que les coastlines correspondent (sinon ça « bave », testez pour voir !

In [ ]:
# définissons quelques constantes / paramètres :

projPC = crs.PlateCarree()
lonW = -10
lonE = 50
latS = 30
latN = 75
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
projStr = crs.Stereographic(central_longitude=cLon, central_latitude=cLat)
res = '50m'

# traçons notre figure avec ces paramètres :

fig = plt.figure(figsize=(11, 8.5))
ax = plt.axes(projection=projStr)
ax.set_title('Projection stéréographique')

gl = ax.gridlines(
    draw_labels=True, 
    linewidth=2, 
    color='gray', 
    alpha=0.5, 
    linestyle='--'
)

ax.set_extent([lonW, lonE, latS, latN], crs=projPC)

ax.coastlines(resolution=res, color='black')
ax.set_facecolor(feature.COLORS['water']) # on colore la mer
ax.add_feature(feature.LAND.with_scale(res)) # on colore la terre
ax.add_feature(feature.BORDERS.with_scale(res), 
                                linewidth=0.75, 
                                edgecolor='red', 
                                linestyle='--')
ax.add_feature(feature.LAKES.with_scale(res), alpha=0.8)
ax.add_feature(feature.RIVERS.with_scale(res), 
                                alpha=0.8, 
                                linewidth=0.5, 
                                edgecolor='blue');

#### Exercice

Explorez les différents paramètres vu ci-dessus pour bien comprendre leurs effets (notamment la résolution, utilisez des résolutions différntes, etc.).

In [ ]:
# Votre code ici


#### Ajoutez des données

Notre objectif est d’ajouter des données sur les fonds de carte que nous avons créé. Nous allons pour cela la méthode `.add_geometries()` où les données que nous désirons affichées seront passé à l’arguement `array`. Le reste (notamment la barre de couleur pour la légende) est du matplotlib classique.

Mais il faut auparavant s’assurer que le `.crs` du geodataframe est bien dans un système de projection. Nous allons le redéfinir avec `crs.PlateCarree()`


In [ ]:
afriquemap_df.crs

In [ ]:
afriquemap_df = afriquemap_df.to_crs(crs.PlateCarree())
afriquemap_df.crs

In [ ]:
import matplotlib.colors as mcolors

projPC = crs.PlateCarree()
lonW = -20
lonE = 60
latS = -40
latN = 40
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
projStr = crs.Stereographic(central_longitude=cLon, central_latitude=cLat)
res = '50m'


fig = plt.figure(figsize=(8, 6.5))
ax = plt.axes(projection=projStr)
ax.set_title('Population des pays d’Afrique')

gl = ax.gridlines(
    draw_labels=True, 
    linewidth=2, 
    color='gray', 
    alpha=0.5, 
    linestyle='--'
)

ax.set_extent([lonW, lonE, latS, latN], crs=projPC)

ax.coastlines(resolution=res, color='black')
ax.set_facecolor(feature.COLORS['water']) # on colore la mer
ax.add_feature(feature.LAND.with_scale(res)) # on colore la terre
ax.add_feature(feature.BORDERS.with_scale(res), 
                                linewidth=0.75, 
                                edgecolor='red', 
                                linestyle='--')
ax.add_feature(feature.LAKES.with_scale(res), alpha=0.8)
ax.add_feature(feature.RIVERS.with_scale(res), 
                               alpha=0.8, 
                               linewidth=0.5, 
                               edgecolor='blue')

art = ax.add_geometries(afriquemap_df.geometry, 
                        crs=afriquemap_df.crs,
                        array=afriquemap_df.POP_EST,
                        edgecolor='black',
                        cmap='YlGnBu',
                        norm=mcolors.LogNorm(vmin=1e6))
cbar = fig.colorbar(art, orientation='vertical', extend='min')
cbar.set_label('Number of people')

Pour aller plus loin, et si vous rencontrez des problèmes d’import des données (format de fichier, etc.), n’hésitez pas à regarder la documentation de cartopy sur les entrées/sorties (modules cartopy.io, et notamment cartopy.io.shapereader).

### Folium

`pip install folium`

Folium est une bibliothèque qui peut apporter beaucoup d’interaction, basée sur la bibliothèque javascrip Leaflet (rappel : il inverse les lat/long en long/lat).

Folium permet très rapidement d’avoir un fond de carte, connecté avec un diffuseur de flux comme Open Street Map (mais aussi CartoDB, Stamen, Mapbox…): 

In [ ]:
import folium

In [ ]:
africa_map = folium.Map(location=[afriquemap_df.centroid.y.mean(), 
                                    afriquemap_df.centroid.x.mean()], 
                                    zoom_start=3, 
                                    tiles='openstreetmap')
africa_map

Cette carte de base, vide, peut se voir ajouter des éléments avec la méthode `.add_to()`.

Par exemple, on aurait très bien pu écrire (on en profite pour tester un autre diffuseur de fond de carte) :

In [ ]:
africa_map = folium.Map(location=[afriquemap_df.centroid.y.mean(), 
                                    afriquemap_df.centroid.x.mean()], 
                                    zoom_start=3, 
                                    tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(africa_map)
africa_map

Folium dispose par ailleurs d’une fonction `folium.Choropleth()` pour créer très facilement des carte choroplèthes. Cette fonction prend 3 arguments :
* `geo_data` : l’objet qui contient les géométries (a priori un fichier GeoJSON, mais on peut passer un geodataframe qui sera automatiquement converti en GeoJSON par Folium)
* `data` : le dataframe qui contient les données à afficher
* `columns` : les (2) colonnes qui vont servir à créer la chloroplèthe. Une colonne contient les entités géographiques (pays, régions, villes…) auxquelles se rapportent les mesures (métriques) que l’on doit représenter (la deuxième colonne)
* `key_on` : la clé par laquelle géométrie et entités vont être mises en correspondance pour créer la carte

On voit que cette fonction n’a pas été prévue dans le cas où on a un seul geodataframe qui contient toutes les données (géométries, entitée et données à afficher). En fait le problème vient du paramètre `key_on` qui hérite de contraintes du Leaflet (javascript), il  va falloir l’écrire sur ce format : `feature.properties.<nom_de_la_colonne_des_entités>`, ici cela deonnera donc `feature.properties.ISO_A3` :


In [ ]:
scale = (afriquemap_df['POP_EST'].quantile((0,0.1,0.75,0.9,0.98,1))).tolist()
folium.Choropleth(geo_data=afriquemap_df,
                 name='Population par pays en Afrique',
                 data=afriquemap_df,
                 columns=['ISO_A3','POP_EST'],
                 key_on="feature.properties.ISO_A3",
                 threshold_scale=scale,
                 fill_color='YlGnBu',
                 fill_opacity=1,
                 line_opacity=0.2,
                 legend_name='Population',
                 smooth_factor=0
                ).add_to(africa_map)

africa_map

L’intérêt de folium est qu’il pousse l’interactivité jusqu’à pouvoir ajouter des éléments lorsque l’on survole la carte, ajouter un panneau de contrôle…

#### Information en survolant la carte

Une possibilité três intéressante lorsque l’on rajoute de l’interactivité à une dataviz est de pouvoir disposer d’infobulles qui donnent les valeurs précises des métriques dont on dispose quand on survole un élément ou objet. Ici on a une représentation en couleur de la population de chaque pays, en survolant les pays on aimerait bien disposer d’information plus détaillées, au moins l’indentifiant du pays et la valeur exacte de sa population.

On peut rajouter un tel objet (attention à la syntaxe avec des bouts de javascript/HTML/CSS dedans)

In [ ]:
folium.features.GeoJson(
                    data=afriquemap_df,
                    name='Population par pays en Afrique',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'transparent','fillColor':'transparent'},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['ISO_A3',
                                'POP_EST'
                               ],
                        aliases=["Code pays:",
                                 "Population estimée:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(africa_map) 

africa_map

Exercice : rajouter des champs aux infobulles comme le nom complet du pays, ou d’autres éléments du dataframe qui vous semblent pertinents. Jouez avec les paramètres et liser la doc pour comprendre à quoi ils servent.

In [ ]:
# votre code ici


#### Panneau de contrôle

Folium nous permet aussi de rajouter des panneaux de contrôle, etc. mais cela dépasse le cadre de ce cours.

On peut avoir un premier aperçu ici :

In [ ]:
folium.LayerControl().add_to(africa_map)
africa_map

Enfin on peut aussi sauvegarder notre carte au format html : 

In [ ]:
africa_map.save("africa_pop.html") 

#### Exercice

Créer une carte « PIB ».

Bonus : à l’aide de `.TileLayer()` (voir la doc) superposer les cartes PIB et Population, et autoriser la sélection de l’affichage avec un panneau de contrôle.

### Plotly

Plotly permet aussi de créer des cartes choroplèthes très simplement.
Voir le tutoriel directement dans la doc : https://plotly.com/python/choropleth-maps/

Exercice : recréez les cartes choroplètes que nous avons réalisées ci-dessus à l’aide de Plotly :

In [ ]:

# votre code ici




## Exercice

Plus compliqué : aggrégeons des données de sources différentes ! Essayez dans votre projet si elles se prêtent à la cartographie, sinon cherchez ces données dans les sources indiquées.

1. Trouver des données chiffrées avec info geographique (pays, villes…). Attention, ne pas prendre des données déjà agrégées par pays, régions, etc. c’est le but de l’exercice ;)
2. Merger/groupby ces données pour calculer des scores (moyennes, décompte, pourcentage, ce que vous voulez !) par entitées géographiques, par exemple pays, régions, arrondissements, en fonction de la carte dont vous disposez (il faudra sûrement recoder / renommer des colonnes de réf. pays/région, etc pour pouvoir merger sur un df geospatial)
3. Afficher une carte qui associe un code couleur à chaque valeur du score (palette, « température »)